# What is Sorting?
Sorting is the process of arranging a list of items in a particular order (i.e. ascending, descending, etc.)

In [1]:
from manim import *
from manim_voiceover import VoiceoverScene
from manim_voiceover.services.recorder import RecorderService

Manim Community v0.18.0

Could not import the PyAudio C module 'pyaudio._portaudio'.


[01/17/24 17:11:58] ERROR    Missing packages. Run `pip install "manim-voiceover[recorder]"` to use  ]8;id=701515;file:///Users/pranavdhinakar/opt/anaconda3/envs/manim/lib/python3.9/site-packages/manim_voiceover/services/recorder/__init__.py\__init__.py]8;;\:]8;id=433166;file:///Users/pranavdhinakar/opt/anaconda3/envs/manim/lib/python3.9/site-packages/manim_voiceover/services/recorder/__init__.py#14\14]8;;\
                             RecorderService.                                                                      

In [7]:
# Introducing the problem of sorting
class Introduction(Scene):
    def construct(self):
        title = Text("Sorting Algorithms").scale(1.5)
        self.play(Write(title))
        self.wait(1)
        self.play(FadeOut(title))

        problem = Text("Given a list of numbers, \n sort them in ascending order.").scale(1.5)
        self.play(Write(problem))
        self.wait(1)
        self.play(FadeOut(problem))

        self.wait(1)

%manim -ql -v WARNING Introduction

In [14]:
# Define an array object so that when it is animated it is a long rectangle with small squares inside
class Array(Rectangle):
    def __init__(self, values, **kwargs):
        self.values = values # List of values in the array
        self.n = len(values) # Number of elements in the array
        
        # Call the constructor of the parent class (Rectangle)
        # Ensure the rectangle is large enough to contain all squares
        super().__init__(width=self.n, height=1, **kwargs)

        # Initialize the squares and texts
        self.squares = VGroup()  # Group for squares
        self.texts = VGroup()    # Group for text
        
        # Calculate the side length of each square based on the number of elements
        square_side = self.width / self.n

        # Create squares and texts for each value
        for i, value in enumerate(self.values):
            square = Square(side_length=square_side)
            # Position each square within the rectangle
            square.move_to(self.get_left() + RIGHT * square_side * (0.5 + i))
            self.squares.add(square)

            text = Text(str(value), font_size=36)
            text.move_to(square.get_center())
            self.texts.add(text)

        # Add the squares and texts as submobjects
        self.add(self.squares, self.texts)
        
    def update_array(self, new_values):
        """ Update the array with new values. """
        self.values = new_values
        for text, value in zip(self.texts, self.values):
            text.set_text(str(value))
    

In [20]:
class ArrayScene(Scene):
    def construct(self):
        # Create the array object
        array_values = [5, 3, 1, 2, 4]
        array = Array(array_values)
        
        # Center the array in the scene
        array.move_to(ORIGIN)
        
        # Animate the array
        self.play(Write(array))
        
        self.wait()
    
%manim -ql -v WARNING ArrayScene

In [32]:
class LatexArray(Rectangle):
    def __init__(self, n, **kwargs):
        self.n = n  # Number of elements in the array

        # Call the constructor of the parent class (Rectangle)
        super().__init__(width=self.n, height=1, **kwargs)

        # Initialize the squares and texts
        self.squares = VGroup()  # Group for squares
        self.texts = VGroup()    # Group for text

        # Calculate the side length of each square based on the number of elements
        square_side = self.width / self.n

        # Create squares and LaTeX texts for each value
        for i in range(self.n):
            square = Square(side_length=square_side)
            # Position each square within the rectangle
            square.move_to(self.get_left() + RIGHT * square_side * (0.5 + i))
            self.squares.add(square)

            #if i == 0:
            #    latex_text = f"a_{{{i+1}}}"
            #elif i == self.n - 1:
            #    latex_text = f"a_{{{i+1}}}"
            #else:
            #    latex_text = "\\dots"
            if i == self.n - 2:
                latex_text = "\\dots"
            elif i == self.n - 1:
                latex_text = f"a_{{n}}"
            else:
                latex_text = f"a_{{{i+1}}}"
                
            
            text = MathTex(latex_text, font_size=36)
            text.move_to(square.get_center())
            self.texts.add(text)

        # Add the squares and texts as submobjects
        self.add(self.squares, self.texts)

    def update_array(self, new_values):
        """ Update the array with new values. """
        # Implement the logic to update LaTeX text if needed
        pass


In [33]:
class LatexArrayScene(Scene):
    def construct(self):
        # Create the array object
        array = LatexArray(5)

        # Center the array in the scene
        array.move_to(ORIGIN)

        # Animate the array
        self.play(Write(array))

        self.wait()

%manim -ql -v WARNING LatexArrayScene

In [40]:
# Constructing a scene to introduce the formal definition of the sorting problem
class FormalDefinition(Scene):
    def construct(self):
        
        # Create the definition
        definition = Tex(r"""
        \textbf{Sorting Problem:} \\
        Given a list of numbers $a_1, a_2, \dots, a_n$, \\
        sort them in ascending order.
        """).scale(1.5)
        self.play(Write(definition))
        self.wait(1)
        self.play(FadeOut(definition))
        
        # Move the above text up, fade out the title "Sorting Problem" keep the problem prompt
        definition.shift(UP)
        #self.play(FadeOut(definition[0]))
        self.wait(1)
        
        # Using the Latex Array as this is a definitio of the problem on a mathematical scale
        array = LatexArray(5)
        array.move_to(ORIGIN)
        self.play(Write(array))
        self.wait(1)
        self.play(FadeOut(array))
        
    
        
%manim -ql -v WARNING FormalDefinition

In [65]:
from manim import *

class Gear(VGroup):
    def __init__(self, num_teeth=8, radius=1, tooth_size=0.1, **kwargs):
        super().__init__(**kwargs)
        self.radius = radius  # Store radius as an instance attribute
        self.extended_radius = radius + tooth_size  # Include the tooth size in the extended radius
        
        # Main circle
        circle = Circle(radius=self.radius)
        self.add(circle)

        # Teeth around the circle
        for i in range(num_teeth):
            angle = i * TAU / num_teeth
            # Adding a small rectangle as a tooth
            tooth = Rectangle(height=tooth_size, width=tooth_size*2)
            tooth.set_fill(WHITE, opacity=1)
            tooth.set_stroke(width=0)
            tooth.next_to(circle, UP, buff=0)
            tooth.rotate(angle, about_point=circle.get_center())
            self.add(tooth)

        # Central circle to cover overlapping parts
        central_circle = Circle(radius=tooth_size*1.5)
        central_circle.set_fill(BLACK, opacity=1)
        self.add(central_circle)
        
        self.set_stroke(width=0)
        self.set_fill(GREY_B, opacity=1)


class GearBox(VGroup):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # Create two gears
        self.gear1 = Gear(num_teeth=15, radius=0.6, tooth_size=0.1)  # Larger gear
        self.gear2 = Gear(num_teeth=8, radius=0.3, tooth_size=0.1)   # Smaller gear

        # Calculate the total width and height needed for the box
        buffer_space = 0.4  # Increased buffer space
        total_width = self.gear1.extended_radius + self.gear2.extended_radius + buffer_space
        total_height = total_width  # Making the box square

        # Create the box, adjusting its size based on gears
        box = RoundedRectangle(corner_radius=0.2, height=total_height+0.2, width=total_width+0.2)
        self.add(box)

        # Position gears in the desired corners
        self.gear1.move_to(box.get_center() + LEFT * self.gear1.extended_radius / 2 + DOWN * self.gear1.extended_radius / 2)
        self.gear2.move_to(box.get_center() + RIGHT * self.gear2.extended_radius / 2 + UP * self.gear2.extended_radius / 2)

        self.add(self.gear1, self.gear2)

        # Define the updater function for gear rotation
        self.gear1.add_updater(lambda m, dt: m.rotate(TAU/8 * dt))
        self.gear2.add_updater(lambda m, dt: m.rotate(-TAU/8 * dt * self.gear1.radius / self.gear2.radius))

        box.shift(LEFT * 0.2)  # shift the box to the left and down a bit
        box.shift(DOWN * 0.2)  # shift the box to the left and down a bit

        
class GearBoxScene(Scene):
    def construct(self):
        gearbox = GearBox()        
        self.play(FadeIn(gearbox))
        self.wait(5)
        gearbox.gear1.clear_updaters()
        gearbox.gear2.clear_updaters()
        self.wait(1)

# To run this scene, use the following command in your terminal:
%manim -ql -v WARNING GearBoxScene

# Scene Structure

## Scene 1: Introduction
Voiceover: "What is an algorithm?"
Tex: "Algorithm"

In [92]:
class OpeningScene(Scene):
    def construct(self):
        # First, what is an Algorithm?
        tx1 = Text("Algorithm").scale(1.0)
        self.play(Write(tx1))
        self.wait(1)
        
        # Move the text up to the top of the scene
        self.play(tx1.animate.shift(UP*3))
        self.wait(1)
        
        # Introduce the concept of Inputs
        input_text = Text("Inputs", color=BLUE).next_to(tx1, DOWN * 0.8, buff=1)
        self.play(Write(input_text))

        # Create input circles
        inputs = VGroup(*[Circle(color=BLUE).scale(0.2) for _ in range(3)])
        inputs.arrange(RIGHT, buff=0.5)
        inputs.next_to(input_text, DOWN)

        # Create square brackets and variable label
        left_bracket = Text("[", color=WHITE).scale(1.5)
        right_bracket = Text("]", color=WHITE).scale(1.5)
        variable_label = Text("x =", color=WHITE).scale(0.7)

        # Position the brackets and label
        left_bracket.next_to(inputs, LEFT, buff=0.1)
        right_bracket.next_to(inputs, RIGHT, buff=0.1)
        variable_label.next_to(left_bracket, LEFT, buff=0.2).align_to(left_bracket, DOWN)

        # Group the circles with the brackets
        input_group = VGroup(left_bracket, inputs, right_bracket)
        input_group.next_to(input_text, DOWN)

        # Animate the creation of the inputs and brackets
        self.play(LaggedStart(*[DrawBorderThenFill(obj) for obj in inputs], lag_ratio=0.5))
        self.play(Write(left_bracket), Write(right_bracket), Write(variable_label))

        
        # Introduce the GearBox (Algorithm at Work)
        gearbox = GearBox()
        # Move the gear box to just below the inputs
        gearbox.move_to(input_text.get_center() + DOWN*3.0)
        
        self.play(FadeIn(gearbox))
        #self.play(Draw(gearbox))

        self.wait(1)

        # Animate Inputs Moving Towards the GearBox
        #for circle in inputs:
        #    self.play(circle.animate.move_to(gearbox.get_center() + UP*0.5))
        #    # Fade out each circle as it reaches the gear box
        #    self.play(FadeOut(circle))
        #self.wait(1)
        
        #self.wait(2)

%manim -qh -v WARNING OpeningScene